# Chapter 62: Edge Computing and CI/CD

While data center deployments benefit from abundant compute, reliable high-bandwidth networking, and physical security, edge computing operates under opposing constraints: devices may possess only ARM-based single-board computers with 512MB RAM, connectivity might be intermittent 4G/LTE or LoRaWAN with strict data caps, and hardware resides in unsecured retail stores, factory floors, or vehicles traversing cellular dead zones. Yet these environments increasingly require cloud-native capabilities—container orchestration, automated rollouts, and GitOps workflows—to process data locally for latency-sensitive applications (autonomous driving, industrial IoT, real-time fraud detection) and comply with data sovereignty regulations. This chapter examines how to adapt Kubernetes and CI/CD practices for resource-constrained, geographically distributed edge fleets, addressing multi-architecture container builds, delta update mechanisms to conserve bandwidth, and security postures for devices operating beyond traditional network perimeters.

## 62.1 Edge Computing Overview

Edge computing positions computational resources proximal to data sources, reducing round-trip latency to centralized clouds and enabling autonomous operation during network partitions. Unlike cloud data centers where infrastructure homogeneity simplifies deployment, edge environments exhibit extreme heterogeneity—mixing x86 industrial PCs, ARM64 NVIDIA Jetson devices, and RISC-V microcontrollers—often managed in fleets numbering tens of thousands across disparate geographic locations.

### Edge Deployment Patterns

**Heavy Edge (Far Edge):**
Runs full Kubernetes clusters on robust hardware (factory servers, retail backrooms, cell towers). Characteristics include: 16-64GB RAM, persistent power, wired ethernet with intermittent cloud connectivity, managed via GitOps with local control planes.

**Light Edge (Near Edge):**
Single-node Kubernetes or container runtimes on constrained devices (kiosks, drones, sensors). Characteristics include: 512MB-4GB RAM, ARM processors, battery or solar power, wireless connectivity, requiring ultra-efficient resource utilization.

**IoT Edge:**
 microcontroller-class devices running containerd or WebAssembly runtimes rather than full Kubernetes. Managed via fleet orchestrators (Azure IoT Edge, AWS Greengrass) rather than traditional CI/CD pipelines.

### CI/CD Challenges at the Edge

**Bandwidth Constraints:**
Pushing 500MB container images to 10,000 devices over cellular networks is economically and technically infeasible. Delta updates, compression, and layer caching become critical.

**Intermittent Connectivity:**
Devices may be offline for hours or days. Rollouts must be asynchronous, resumable, and tolerant of mid-update connection drops without bricking devices.

**Resource Heterogeneity:**
Images must support multiple architectures (amd64, arm64, arm/v7) and varying CPU instruction sets (AVX2 vs. NEON).

**Security Posture:**
Devices operate in physically unsecured locations, requiring encrypted storage, hardware security modules (HSM), and mutual TLS with short-lived certificates.

## 62.2 Lightweight Kubernetes Distributions

Standard Kubernetes control planes require 2GB+ RAM and multiple nodes—prohibitive for edge deployment. Lightweight distributions strip control plane components to essentials, enabling single-node operation on resource-constrained hardware.

### K3s

Rancher Labs' K3s is a certified Kubernetes distribution weighing under 100MB, optimized for ARM and x86 edge architectures. It replaces etcd with SQLite (or external databases for HA), bundles Flannel CNI, and removes alpha features and legacy cloud providers.

**Architecture:**
- **Single Binary:** 40MB binary containing apiserver, scheduler, controller-manager, and flannel.
- **SQLite Backend:** Default storage eliminates etcd memory overhead (~1GB savings).
- **Containerd Default:** Uses containerd directly, not Docker, reducing daemon overhead.
- **ARM Support:** First-class support for Raspberry Pi, NVIDIA Jetson, and ARM64 servers.

**Installation (Edge Device):**
```bash
# Install K3s on edge device (ARM64 or x86)
curl -sfL https://get.k3s.io | INSTALL_K3S_EXEC="server --disable traefik --write-kubeconfig-mode 644" sh -

# Verify
kubectl get nodes
# NAME        STATUS   ROLES                  AGE   VERSION
# edge-node-1 Ready    control-plane,master   10s   v1.28.3+k3s1
```

**GitOps Agent Configuration:**
For edge fleets, run K3s in agent mode connecting to cloud-hosted control planes (K3s "servers") or use fully autonomous single-node mode with local GitOps operators:

```yaml
# /etc/rancher/k3s/config.yaml (Agent mode)
server: https://control-plane.company.com:6443
token: ${K3S_TOKEN}
node-label:
  - "edge.location=store-123"
  - "edge.tier=far-edge"
kubelet-arg:
  - "system-reserved=cpu=100m,memory=200Mi"
  - "kube-reserved=cpu=100m,memory=200Mi"
```

### MicroK8s

Canonical's MicroK8s offers snap-based installation with strict confinement, suitable for Ubuntu Core IoT deployments:

```bash
# Install on Ubuntu Core
snap install microk8s --classic --channel=1.28/stable

# Enable only required addons (resource conservation)
microk8s enable dns
microk8s enable storage
microk8s enable registry

# Join existing cluster (for multi-node edge sites)
microk8s join 192.168.1.100:25000/${TOKEN}
```

**Resource Profiling:**
MicroK8s can operate with as little as 540MB RAM when optimized:
```bash
microk8s status
# Disable unused services
microk8s disable metrics-server  # If external monitoring used
microK8s disable dashboard       # If not needed locally
```

### K3os and Immutable Edge OS

K3os combines Linux OS and K3s into an immutable image, enabling atomic updates and rollbacks at the OS level:

```yaml
# cloud-config.yml for K3os installation
ssh_authorized_keys:
  - ssh-rsa AAAAB3NzaC1...

k3os:
  password: changeme
  server_url: https://control-plane.company.com:6443
  token: ${CLUSTER_TOKEN}
  labels:
    - region=north-america
    - site=retail-store-456
  k3s_args:
    - agent
    - "--node-label"
    - "edge-type=pos-terminal"
```

### Comparison Matrix

| Distribution | Min RAM | Architecture | Storage Backend | Best Use Case |
|-------------|---------|--------------|-----------------|---------------|
| **K3s** | 512MB | x86_64, ARM64, ARMv7 | SQLite/Postgres | General edge, industrial IoT |
| **MicroK8s** | 540MB | x86_64, ARM64 | dqlite | Ubuntu Core, development |
| **k0s** | 1GB | x86_64, ARM64 | SQLite | Telco edge, multi-node sites |
| **KubeEdge** | 256MB (edgecore) | x86_64, ARM64 | SQLite | Cloud-edge collaboration |

## 62.3 Container Optimization for Edge

Standard container images (Ubuntu, Debian-based) consume 100MB+ before application code. Edge devices require aggressive optimization to minimize pull times and storage usage.

### Multi-Architecture Builds

Edge fleets mix architectures (x86 servers with ARM sensors). CI pipelines must produce multi-arch manifests:

```dockerfile
# Dockerfile.optimized
FROM --platform=$BUILDPLATFORM golang:1.21 AS builder
WORKDIR /app
COPY go.mod go.sum ./
RUN go mod download
COPY . .
ARG TARGETOS
ARG TARGETARCH
RUN CGO_ENABLED=0 GOOS=$TARGETOS GOARCH=$TARGETARCH go build -ldflags="-w -s" -o edge-app

# Final stage - distroless for security and size
FROM gcr.io/distroless/static:nonroot-${TARGETARCH}
COPY --from=builder /app/edge-app /edge-app
USER 65532:65532
ENTRYPOINT ["/edge-app"]
```

**Buildx Configuration:**
```bash
# Create builder with emulation support
docker buildx create --name multiarch --use --bootstrap
docker run --privileged --rm tonistiigi/binfmt --install all

# Build and push multi-arch manifest
docker buildx build \
  --platform linux/amd64,linux/arm64,linux/arm/v7 \
  -t registry.company.com/edge-app:v1.2.0 \
  --push \
  -f Dockerfile.optimized .
```

**Manifest Inspection:**
```bash
docker manifest inspect registry.company.com/edge-app:v1.2.0
# Shows manifests for amd64, arm64, arm/v7
```

### Distroless and Scratch Images

Eliminate package managers, shells, and OS utilities to reduce attack surface and size:

```dockerfile
# For statically compiled Go/Rust apps
FROM scratch
COPY --from=builder /app/edge-app /edge-app
COPY --from=builder /etc/ssl/certs/ca-certificates.crt /etc/ssl/certs/
USER 65532
ENTRYPOINT ["/edge-app"]
```

**Size Comparison:**
- Ubuntu base: ~120MB
- Alpine: ~15MB
- Distroless: ~5MB (application binary only)
- Scratch: ~2MB (binary + certs only)

### Layer Optimization

Structure Dockerfile to maximize cache hits on edge devices with limited bandwidth:

```dockerfile
# Bad: Dependencies change forces full rebuild
COPY . /app
RUN go build

# Good: Separate dependency layer (cached)
COPY go.mod go.sum ./
RUN go mod download  # Cached layer
COPY . .
RUN go build
```

**Image Pull Policies:**
Configure K3s/MicroK8s to cache aggressively:
```yaml
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: app
    image: registry.company.com/edge-app:v1.2.0
    imagePullPolicy: IfNotPresent  # Use local cache if available
```

## 62.4 GitOps for Fleet Management

Managing thousands of distributed edge devices requires GitOps patterns that tolerate intermittent connectivity and support progressive rollouts across geographic tiers.

### Fleet Architecture

**Hub-and-Spoke Model:**
Cloud-hosted GitOps control plane (ArgoCD/Flux) manages regional "hub" clusters, which in turn manage local "spoke" edge devices via constrained connectivity.

**Autonomous Edge Model:**
Each edge device runs a local GitOps agent (Flux agent, ArgoCD agent) that pulls from cached Git repositories or local Git servers when cloud connectivity is unavailable.

### Flux for Edge

Flux v2's lightweight agents suit edge deployment:

```yaml
# Install Flux on edge device (lightweight)
flux install --namespace=flux-system --components=source-controller,kustomize-controller

# GitRepository with shortened sync interval for edge
apiVersion: source.toolkit.fluxcd.io/v1
kind: GitRepository
metadata:
  name: edge-config
  namespace: flux-system
spec:
  interval: 5m  # Longer interval to conserve bandwidth
  url: https://github.com/company/edge-configs.git
  ref:
    branch: main
  secretRef:
    name: git-credentials
  # Verify commits for security
  verify:
    mode: head
    secretRef:
      name: git-pgp-public-keys
---
apiVersion: kustomize.toolkit.fluxcd.io/v1
kind: Kustomization
metadata:
  name: edge-apps
spec:
  interval: 10m
  path: ./overlays/store-123
  prune: true
  sourceRef:
    kind: GitRepository
    name: edge-config
  healthChecks:
  - apiVersion: apps/v1
    kind: Deployment
    name: pos-terminal
    namespace: retail
  timeout: 2m  # Shorter timeout for edge responsiveness
```

### ArgoCD Edge Agent

ArgoCD's ApplicationSet controller manages fleets via generators:

```yaml
apiVersion: argoproj.io/v1alpha1
kind: ApplicationSet
metadata:
  name: edge-fleet
spec:
  generators:
  - list:
      elements:
      - cluster: store-001
        url: https://192.168.1.101:6443
        region: us-west
      - cluster: store-002
        url: https://192.168.1.102:6443
        region: us-west
      # ... thousands of stores
  template:
    metadata:
      name: '{{cluster}}-pos-system'
    spec:
      project: edge-deployments
      source:
        repoURL: https://github.com/company/edge-apps.git
        targetRevision: HEAD
        path: pos-system/overlays/{{region}}
      destination:
        server: '{{url}}'
        namespace: retail
      syncPolicy:
        automated:
          prune: true
          selfHeal: true
        syncOptions:
        - PruneLast=true
        - CreateNamespace=true
      ignoreDifferences:  # Handle edge-specific config drift
      - group: apps
        kind: Deployment
        jsonPointers:
        - /spec/replicas  # Allow HPA to manage replicas
```

### Progressive Rollouts Across Fleet

Use Flagger or Argo Rollouts with geographic canarying:

```yaml
apiVersion: flagger.app/v1beta1
kind: Canary
metadata:
  name: edge-app
  namespace: retail
spec:
  targetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: pos-terminal
  service:
    port: 8080
  analysis:
    interval: 5m  # Longer intervals for edge
    threshold: 3
    maxWeight: 50
    stepWeight: 25
    metrics:
    - name: request-success-rate
      thresholdRange:
        min: 95
    webhooks:
    # Verify connectivity before promotion
    - name: connectivity-check
      type: confirm-rollout
      url: http://flagger-loadtester.test/
      timeout: 30s
      metadata:
        cmd: "curl -sf http://pos-terminal.retail:8080/health"
```

## 62.5 Delta Updates and Bandwidth Optimization

Pushing full container images (100MB+) to thousands of edge devices over metered cellular connections is cost-prohibitive. Delta update mechanisms transmit only changed layers or binary diffs.

### Image Layer Deduplication

Container registries support layer sharing, but edge devices require additional optimization:

**CAS (Content Addressable Storage) Proxy:**
Deploy registry proxies at regional hubs that cache layers and serve edge requests locally:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: registry-cache
spec:
  template:
    spec:
      containers:
      - name: registry
        image: registry:2
        env:
        - name: REGISTRY_PROXY_REMOTEURL
          value: https://registry.company.com
        - name: REGISTRY_STORAGE_FILESYSTEM_ROOTDIRECTORY
          value: /var/lib/registry
        volumeMounts:
        - name: cache
          mountPath: /var/lib/registry
      volumes:
      - name: cache
        hostPath:
          path: /opt/registry-cache  # Persistent local cache
```

### Binary Delta Updates with OSTree

For OS-level updates (K3s upgrades, OS patches), OSTree provides Git-like content tracking with binary deltas:

```bash
# On build server
ostree commit --branch=edge-os/1.2.0 --tree=dir=/buildroot

# Generate delta from previous version
ostree static-delta generate --from=edge-os/1.1.0 --to=edge-os/1.2.0

# Edge device applies delta (downloads ~5MB vs 500MB full image)
ostree pull --delta edge-os/1.2.0
ostree admin deploy edge-os/1.2.0
```

### Containerd Image Pull Optimization

Enable lazy pulling (eStargz) to start containers before full download:

```toml
# /etc/containerd/config.toml
[plugins."io.containerd.grpc.v1.cri".registry]
  [plugins."io.containerd.grpc.v1.cri".registry.mirrors]
    [plugins."io.containerd.grpc.v1.cri".registry.mirrors."docker.io"]
      endpoint = ["https://mirror.gcr.io", "https://registry-1.docker.io"]

[plugins."io.containerd.snapshotter.v1.stargz]
  # Enable eStargz for lazy pulling
```

### Compressed Updates with Zstd

Use Zstd compression for image layers (faster decompression than gzip on ARM):

```dockerfile
# Build with BuildKit and zstd
docker buildx build \
  --output type=image,compression=zstd,force-compression=true \
  -t registry.company.com/edge-app:v1.2.0 \
  --push .
```

## 62.6 Security for Edge Deployments

Edge devices operate outside physical security perimeters, requiring zero-trust architectures, hardware-backed identity, and encrypted storage.

### Device Identity and Attestation

**SPIFFE/SPIRE:**
Issue short-lived X.509 certificates to edge workloads based on hardware attestation:

```yaml
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: spire-agent
spec:
  template:
    spec:
      containers:
      - name: spire-agent
        image: ghcr.io/spiffe/spire-agent:1.8.0
        volumeMounts:
        - name: spire-socket
          mountPath: /run/spire/sockets
        args:
        - -config
        - /run/spire/config/agent.conf
      volumes:
      - name: spire-socket
        hostPath:
          path: /run/spire/sockets
          type: DirectoryOrCreate
---
# Workload registration with TPM attestation
apiVersion: spire.spiffe.io/v1alpha1
kind: ClusterSPIFFEID
metadata:
  name: edge-pos-terminal
spec:
  spiffeIDTemplate: "spiffe://company.com/edge/{{.Node.Labels.site}}/pos"
  podSelector:
    matchLabels:
      app: pos-terminal
  nodeSelector:
    matchLabels:
      edge-type: far-edge
  # Require TPM attestation
  federatesWith: []
```

**TPM (Trusted Platform Module):**
Store device private keys in TPM chips to prevent extraction even if device is physically compromised:

```bash
# Generate key in TPM
tpm2tss-genkey -a rsa -s 2048 device_key.tss

# Use with K3s
k3s server --token-file /dev/tpm0  # Token sealed in TPM
```

### Network Security

**mTLS with Cert-Manager:**
Automatically rotate certificates for edge devices:

```yaml
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: edge-device-cert
  namespace: edge-system
spec:
  secretName: edge-tls
  issuerRef:
    name: step-issuer  # Smallstep CA for edge
    kind: StepIssuer
  commonName: "edge-device-{{.Values.siteId}}.company.com"
  dnsNames:
  - "edge-device-{{.Values.siteId}}.local"
  duration: 24h
  renewBefore: 1h  # Short-lived for edge security
```

**WireGuard for Edge Connectivity:**
Encrypted tunnels between edge and cloud over public internet:

```yaml
apiVersion: vpn.wireguard.io/v1alpha1
kind: WireGuardPeer
metadata:
  name: edge-tunnel
spec:
  endpoint: cloud-vpn.company.com:51820
  allowedIPs:
  - 10.0.0.0/8
  persistentKeepalive: 25  # Keep NAT mapping alive
```

### Image Signing and Verification

Prevent malicious image execution on edge devices:

```bash
# Sign image with Cosign
cosign sign --key cosign.key registry.company.com/edge-app:v1.2.0

# Verify on edge device before deployment
cosign verify --key cosign.pub registry.company.com/edge-app:v1.2.0
```

**Admission Control:**
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sVerifiedImage
metadata:
  name: verify-edge-images
spec:
  match:
    kinds:
    - apiGroups: [""]
      kinds: ["Pod"]
    namespaces: ["edge-apps"]
  parameters:
    allowedRegistries:
    - registry.company.com
    requireSignature: true
```

## 62.7 A/B Testing and Canary Deployments at Edge

Edge canary deployments must account for geographic isolation—rolling out to "Store 001" shouldn't affect "Store 002" due to varying network conditions and local regulations.

### Geographic Canary Strategy

```yaml
apiVersion: flagger.app/v1beta1
kind: Canary
metadata:
  name: edge-pos-update
spec:
  targetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: pos-terminal
  analysis:
    interval: 10m  # Extended for edge stability observation
    threshold: 5
    maxWeight: 30  # Conservative max at edge
    stepWeight: 10
    metrics:
    - name: transaction-error-rate
      thresholdRange:
        max: 0.01  # 1% max error rate for POS
    - name: latency-p99
      thresholdRange:
        max: 500  # 500ms max for payment processing
    webhooks:
    - name: local-health-check
      type: confirm-promotion
      url: http://localhost:8080/health  # Local endpoint only
      timeout: 30s
```

### Feature Flags for Edge

Use lightweight feature flagging (Unleash, Flipt) to enable features without redeployment:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: feature-flags
data:
  config.json: |
    {
      "flags": {
        "new-payment-flow": {
          "enabled": true,
          "strategies": [
            {
              "name": "siteId",
              "parameters": {
                "siteIds": "store-001,store-002"  # Gradual rollout
              }
            }
          ]
        }
      }
    }
```

### Rollback Mechanisms

Edge rollbacks must succeed without cloud connectivity:

**Local Helm Rollback:**
```bash
# Store last 3 revisions locally on device
helm rollback edge-app 2 --namespace retail

# Or use K3s etcd snapshots for full system restore
k3s etcd-snapshot restore --snapshot-retention=3
```

**Blue/Green at Edge:**
Maintain two partitions (A/B) on edge storage:
```yaml
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: app
    image: registry.company.com/edge-app:stable  # Partition A
    volumeMounts:
    - name: rollback-volume
      mountPath: /backup
  - name: watchdog
    image: watchdog:latest
    env:
    - name: ROLLBACK_COMMAND
      value: "kubectl set image deployment/app app=registry.company.com/edge-app:previous"
```

## 62.8 Observability in Edge Environments

Centralized monitoring fails when edges disconnect. Observability must be resilient, storing metrics locally and batching uploads during connectivity windows.

### Edge-Native Monitoring

**Prometheus Remote Write with Buffering:**
```yaml
# Prometheus configuration for edge
remote_write:
  - url: https://prometheus.company.com/api/v1/write
    queue_config:
      max_samples_per_send: 100  # Small batches for intermittent links
      max_shards: 2
      capacity: 2500
    write_relabel_configs:
      - source_labels: [__name__]
        regex: 'edge_.*'
        action: keep  # Only forward edge-specific metrics
```

**Grafana Agent (Lightweight):**
```yaml
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: grafana-agent
spec:
  template:
    spec:
      containers:
      - name: agent
        image: grafana/agent:v0.39.0
        args:
        - -config.file=/etc/agent/agent.yaml
        - -enable-features=remote-configs
        volumeMounts:
        - name: wal
          mountPath: /var/lib/agent
      volumes:
      - name: wal
        hostPath:
          path: /opt/agent-wal  # Persist across restarts
```

### Distributed Tracing with Batching

```yaml
# OpenTelemetry Collector for edge
apiVersion: opentelemetry.io/v1alpha1
kind: OpenTelemetryCollector
metadata:
  name: edge-collector
spec:
  mode: deployment
  config: |
    receivers:
      otlp:
        protocols:
          grpc:
            endpoint: 0.0.0.0:4317
    processors:
      batch:
        timeout: 60s  # Batch for 60s or until connection available
        send_batch_size: 1024
      resource:
        attributes:
        - key: edge.site
          value: store-123
          action: upsert
    exporters:
      otlphttp:
        endpoint: https://tempo.company.com
        tls:
          insecure: false
      file:  # Local backup if cloud unreachable
        path: /var/log/traces/backup.json
    service:
      pipelines:
        traces:
          receivers: [otlp]
          processors: [batch, resource]
          exporters: [otlphttp, file]
```

### Log Shipping with Fluent Bit

Lightweight log processor optimized for edge:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: fluent-bit-config
data:
  fluent-bit.conf: |
    [INPUT]
        Name              tail
        Tag               edge.app
        Path              /var/log/containers/*.log
        Parser            docker
        Mem_Buf_Limit     5MB  # Memory limit for buffering
    
    [OUTPUT]
        Name              forward
        Match             *
        Host              cloud-aggregator.company.com
        Port              24224
        tls               On
        tls.verify        On
        Retry_Limit       10
        storage.total_limit_size  100M  # Disk buffer when offline
```

---

## Chapter Summary and Preview

This chapter explored the unique challenges of extending CI/CD pipelines to edge computing environments, where resource constraints, intermittent connectivity, and physical security concerns dominate architectural decisions. We established that lightweight Kubernetes distributions—K3s, MicroK8s, and K3os—enable container orchestration on hardware as minimal as ARM64 single-board computers with 512MB RAM, replacing etcd with SQLite and eliminating control plane overhead.

Container optimization emerged as critical for bandwidth-constrained deployment, requiring multi-architecture builds (amd64, arm64, arm/v7), distroless base images reducing footprints from 100MB to 5MB, and layer caching strategies to maximize reuse across fleet updates. GitOps patterns adapted for edge tolerate intermittent connectivity through asynchronous reconciliation agents (Flux, ArgoCD) running locally on devices, with regional hub caches reducing redundant data transfer across cellular networks.

Delta update mechanisms—including OSTree for atomic OS updates, Zstd compression for image layers, and eStargz lazy pulling—minimize bandwidth consumption when pushing updates to thousands of geographically distributed devices. Security postures for edge require hardware-backed identity (TPM attestation, SPIFFE/SPIRE short-lived certificates), mutual TLS with automated rotation, and admission controllers verifying image signatures before execution on physically accessible hardware.

A/B testing strategies at the edge emphasize geographic isolation and conservative canary percentages (10-30% vs 50%+ in cloud), with local health verification and automatic rollback capabilities that function autonomously during cloud partitions. Observability shifts to store-and-forward architectures, with Prometheus remote write buffering, OpenTelemetry trace batching, and Fluent Bit log shipping designed to survive extended offline periods without data loss.

**Key Takeaways:**
- Use K3s for edge Kubernetes due to its SQLite backend and ARM support; disable Traefik and metrics-server if not required to save resources
- Build multi-arch images using docker buildx with distroless or scratch bases to minimize pull times and storage on edge devices
- Implement regional registry caches to serve image layers locally, reducing cellular bandwidth costs by 90%+
- Never rely on continuous cloud connectivity; design GitOps agents to reconcile from local Git mirrors or cached states during partitions
- Enforce image signing (Cosign, Notary) with admission control; edge devices are physically accessible and high-risk for tampering
- Use short-lived certificates (24-hour TTL) with automatic rotation via cert-manager or SPIRE; compromise of edge credentials must not grant long-term access
- Implement blue/green partitions on edge storage for instant rollback without network dependency
- Buffer all observability data (metrics, logs, traces) to local disk with size limits to prevent storage exhaustion during extended outages

**Next Chapter Preview:**
This completes Part XII: Advanced Topics. The following section, Part XI: Real-World Projects, transitions from theoretical concepts to practical implementation through four comprehensive projects. Project 1: Simple Web Application CI/CD will guide you through building a complete pipeline for a containerized web application using GitHub Actions, Docker, and Kubernetes. Project 2: Microservices Architecture demonstrates service mesh implementation with Istio, distributed tracing, and canary deployments across multiple services. Project 3: Multi-Environment Enterprise Application covers Helm charts, GitOps with ArgoCD, security hardening, and disaster recovery for a production-grade deployment. Project 4: Database-Intensive Application focuses on stateful workload management, database migration strategies, and backup automation within CI/CD pipelines. These projects synthesize the principles covered throughout this handbook into actionable, industry-standard implementations.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='61. mobile_app_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
